# Introduction

The notebook is intended to predict the chance failure of a specific product code given its attributes and measurements.

This is the [dataset used](https://www.kaggle.com/competitions/tabular-playground-series-aug-2022).

In [ ]:
# Import Standard Libraries
import pandas as pd
import numpy as np

import plotly.express as px
from plotly.io import read_json

# Read Data

In [ ]:
# Read train & test data
train_data = pd.read_csv('./../../data/2022_08/train.csv')
test_data = pd.read_csv('./../../data/2022_08/test.csv')

In [ ]:
train_data.info()

In [ ]:
train_data.head()

# Exploratory Data Analysis

## id

In [ ]:
# Check for duplicates
print('ID duplicates: {}'.format(len(train_data[train_data['id'].duplicated()])))

## Product Code

In [ ]:
# Check for NaN values
print('Product Code NaN values: {}'.format(len(train_data[train_data['product_code'].isna()])))

In [ ]:
# Plot the Histogram of 'product_code' Distribution
figure = px.histogram(train_data, 
                      x='product_code', 
                      title='Product Code Distribution', 
                      labels={'product_code':'Product Code',
                              'count': 'Share'},
                      color_discrete_sequence=['darkgreen'],
                      height=500,
                      histnorm='',
                      template='plotly_dark')

figure.update_layout(yaxis_title='Share', 
                     font=dict(family="PT Sans", 
                               size=14), 
                     title_font=dict(family="PT Sans",
                                     size=30), 
                     title_x=0.5)

# Save figure
figure.write_json("./plots/product_code_distribution.json")

# Read & plot figure
read_json('./plots/product_code_distribution.json').show()

## loading

In [ ]:
# Check for NaN values
print('Loading NaN values: {}'.format(len(train_data[train_data['loading'].isna()])))

In [ ]:
# Plot boxplot of 'loading'
figure = px.box(train_data, 
                x='loading', 
                title='Loading Distribution',
                color_discrete_sequence=['darkgreen'],
                template='plotly_dark')

figure.update_layout(font=dict(family="PT Sans", 
                               size=14), 
                     title_font=dict(family="PT Sans",
                                     size=30), 
                     title_x=0.5)

# Save figure
figure.write_json("./plots/loading_distribution.json")

# Read & plot figure
read_json('./plots/loading_distribution.json').show()

Fill NaN values and clean outliers.

In [ ]:
# Plot the 'loading' distribution with respect to the 'product_code'
figure = px.box(train_data, 
                x='product_code', 
                y='loading',
                points='all',
                color='product_code',
                title='Loading Distribution per Product Code',
                color_discrete_sequence=px.colors.qualitative.Set3,
                height=500,
                template='plotly_dark')

figure.update_layout(font=dict(family="PT Sans", 
                               size=14), 
                     title_font=dict(family="PT Sans",
                                     size=30), 
                     title_x=0.5)

# Save figure
figure.write_json("./plots/loading_product_code_distribution.json")

# Read & plot figure
read_json('./plots/loading_product_code_distribution.json').show()

## Attributes

In [ ]:
# Plot the Histogram of 'vp_region' Distribution with respect to the Year
figure = ex.histogram(vp_users, 
                      x='vp_region', 
                      color='created_at_year',
                      title='Vendor Portal Region Distribution', 
                      labels={'vp_region':'Region',
                              'created_at_year': 'Year'},
                      barmode='group',
                      height=500,
                      histnorm='',
                      category_orders={'created_at_year': [1970, 2019, 2020, 2021, 2022]},
                      template='plotly_dark')

figure.update_layout(yaxis_title='Share', 
                     font=dict(family="PT Sans", 
                               size=14), 
                     title_font=dict(family="PT Sans",
                                     size=30), 
                     title_x=0.7)

# Adjust the x-axis label rotation
figure.update_xaxes(tickangle=45)

figure.show()